In [103]:
import pandas as pd
from scipy import stats
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from tensorflow import keras


ModuleNotFoundError: No module named 'tensorflow'

In [104]:
!pip install tensorflow

     |████████████████████████████████| 217.4 MB 848 bytes/s a 0:00:01  |█▎                              | 8.6 MB 5.7 MB/s eta 0:00:37     |██▋                             | 18.0 MB 6.1 MB/s eta 0:00:33     |███                             | 19.8 MB 6.1 MB/s eta 0:00:33     |██████████▍                     | 70.8 MB 6.5 MB/s eta 0:00:23ta 0:00:23     |████████████                    | 81.0 MB 6.1 MB/s eta 0:00:23     |████████████                    | 81.3 MB 6.1 MB/s eta 0:00:23��████████▊                  | 93.3 MB 25.2 MB/s eta 0:00:05     |██████████████                  | 95.1 MB 25.2 MB/s eta 0:00:05     |███████████████████████████▏    | 184.8 MB 9.1 MB/s eta 0:00:04 0:00:04     |████████████████████████████▎   | 192.1 MB 668 kB/s eta 0:00:38     |█████████████████████████████   | 197.6 MB 454 kB/s eta 0:00:44     |█████████████████████████████▋  | 201.5 MB 6.5 MB/s eta 0:00:03     |███████████████████████████████▏| 211.8 MB 4.1 MB/s eta 0:00:02�███████████▏| 211.9 MB 4.1 MB/s e

In [2]:
def encapsule(df):
    display(df.head(5))
    print( f'shape of df = {df.shape}' )
    return df

In [98]:
def tran(x):
    y = x[1:-1]
    return y.split(',')

def data_wrangle( df ):
    sample = df[['timestamp','raw_sensor_data']];
    sample['parameters'] = sample.raw_sensor_data.apply(tran);
    main = pd.DataFrame( sample.parameters.to_list() , columns = ['v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11'] );
    main['timestamp'] = sample['timestamp'];
    main.drop( 'v1' , axis = 1 , inplace = True );
    main.set_index('timestamp' , inplace=True);
    return main;





In [100]:
def create_segments_and_labels(df, time_steps, step):

    # x, y, z acceleration as features
    N_FEATURES = 10
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['v3'].values[i: i + time_steps]
        ys = df['v4'].values[i: i + time_steps]
        zs = df['v5'].values[i: i + time_steps]
        angle = df['v2'].values[i: i + time_steps]
        u5 = df['v6'].values[i: i + time_steps]
        u6= df['v7'].values[i: i + time_steps]
        u7= df['v8'].values[i: i + time_steps]
        u8= df['v9'].values[i: i + time_steps]
        u9= df['v10'].values[i: i + time_steps]
        u10= df['v11'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        segments.append([xs, ys, zs,angle,u5,u6,u7,u8,u9,u10 ])

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)

    return reshaped_segments

In [91]:
sample = encapsule(pd.read_csv('/Users/arvindchandrasekarreddy/Desktop/PythonProjects/Sord_analysis/csv_vrLLZ7HWebhLTWbGEf0R.csv'))

,id,user_id,email_id,raw_sensor_data,predicted_activity,actual_activity,timestamp,createdAt,updatedAt
0,27967,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[4866.386144,14.911036744980214,2.48,0.28,-9.5...",sit,NaN,2022-02-02T16:13:39.218Z,2022-02-03T03:14:27.006Z,2022-02-03T03:14:27.006Z
1,27968,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[4867.311019,14.911036744980214,2.47,0.27,-9.5...",sit,NaN,2022-02-02T16:13:40.143Z,2022-02-03T03:14:27.006Z,2022-02-03T03:14:27.006Z
2,27969,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[4867.897186,14.911036744980214,2.5,0.24,-9.52...",sit,NaN,2022-02-02T16:13:40.729Z,2022-02-03T03:14:27.006Z,2022-02-03T03:14:27.006Z
3,27970,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[4868.430045,14.911036744980214,2.48,0.26,-9.5...",sit,NaN,2022-02-02T16:13:41.262Z,2022-02-03T03:14:27.006Z,2022-02-03T03:14:27.006Z
4,27971,61f87f357d6217b5ed17047d,imove.mrt@gmail.com,"[4869.016053,14.911036744980214,2.46,0.26,-9.5...",sit,NaN,2022-02-02T16:13:41.848Z,2022-02-03T03:14:27.006Z,2022-02-03T03:14:27.006Z


shape of df = (5000, 9)


In [99]:
main = encapsule(data_wrangle(sample));


,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11
timestamp,,,,,,,,,,
2022-02-02T16:13:39.218Z,14.911036744980214,2.48,0.28,-9.52,-0.0625,-0.0625,-0.125,21.0625,22.375,-40.6875
2022-02-02T16:13:40.143Z,14.911036744980214,2.47,0.27,-9.51,0.125,-0.125,0.125,20.6875,22,-39.375
2022-02-02T16:13:40.729Z,14.911036744980214,2.5,0.24,-9.52,-0.1875,-0.1875,0.125,20.375,21.5625,-40.1875
2022-02-02T16:13:41.262Z,14.911036744980214,2.48,0.26,-9.52,0,0.125,-0.125,21.0625,22.6875,-40.5625
2022-02-02T16:13:41.848Z,14.911036744980214,2.46,0.26,-9.55,-0.1875,-0.1875,-0.25,20.6875,22.375,-40.1875


shape of df = (5000, 10)


In [101]:
create_segments_and_labels(main,35,1)

array([[[  2.48  ,   2.47  ,   2.5   , ...,   2.35  ,   2.48  ,
           2.49  ],
        [  2.45  ,   2.41  ,   2.34  , ...,   2.35  ,   2.38  ,
           2.36  ],
        [  2.38  ,   2.34  ,   2.36  , ...,   2.41  ,   2.36  ,
           2.37  ],
        ...,
        [-40.6875, -40.6875, -39.375 , ..., -41.    , -41.5   ,
         -41.    ],
        [-41.5   , -41.875 , -40.1875, ..., -40.6875, -41.5   ,
         -40.6875],
        [-41.5   , -41.    , -40.1875, ..., -40.6875, -40.1875,
         -40.1875]],

       [[  2.47  ,   2.5   ,   2.48  , ...,   2.48  ,   2.49  ,
           2.45  ],
        [  2.41  ,   2.34  ,   2.39  , ...,   2.38  ,   2.36  ,
           2.38  ],
        [  2.34  ,   2.36  ,   2.4   , ...,   2.36  ,   2.37  ,
           2.37  ],
        ...,
        [-40.6875, -39.375 , -41.5   , ..., -41.5   , -41.    ,
         -41.5   ],
        [-41.875 , -40.1875, -40.6875, ..., -41.5   , -40.6875,
         -41.5   ],
        [-41.    , -40.1875, -40.5625, ..., -40.